In [ ]:
#This module is emplyed to caculate the MACD related parameters.
import pandas
import numpy
#import the module for data processing

In [ ]:
#Define the function to calulate MA
def MA(data,period):
    data=numpy.asarray(data)
    return pandas.ewma(data,float(period))
#Calculate the MA with the indicated period

In [ ]:
#Compute the MACD Signal Crossover line.
def crossover(data,shortperiod,longperiod,signalperiod):
    diff=MA(data,float(shortperiod))-MA(data,float(longperiod))
    signal=MA(diff,signalperiod)
    return diff-signal

In [ ]:
#Get the action at each day based on the index line, e.g., the line can be 
#the MACD Signal Crossover.
def actions(diffsig):
    action=['nochange']
    status=['notholding']
    i=1
    while i<len(diffsig):
        if (diffsig[i]>0) and (diffsig[i-1]<=0) and status[i-1]=='notholding':
            action.append('buy')
            status.append('isholding')
            i+=1
        elif (diffsig[i]<0) and (diffsig[i-1]>=0) and status[i-1]=='isholding':
            action.append('sell')
            status.append('notholding')
            i+=1
        else:
            action.append('nochange')
            status.append(status[i-1])
            i+=1
    return action

In [ ]:
#Given an initial conditon and the action sequence, this function will
#return the value details at each day!
def valuedetail(action,data,initialvalue,commissionrate):
    totalvalue=[0 for i in range(len(data))]
    stockquantity=[0 for i in range(len(data))]
    cash=[0 for i in range(len(data))]
    stockvalue=[0 for i in range(len(data))]
    totalvalue[0]=float(initialvalue)
    stockquantity[0]=0
    cash[0]=float(initialvalue)
    stockvalue[0]=0
    i=1
    while i<len(action):
        if action[i]=='nochange':
            stockquantity[i]=stockquantity[i-1]
            cash[i]=cash[i-1]
            stockvalue[i]=float(data[i])*stockquantity[i]
            totalvalue[i]=stockvalue[i]+cash[i]
        elif action[i]=='sell':
            stockquantity[i]=0
            cash[i]=float(data[i])*stockquantity[i-1]*(1-float(commissionrate))
            stockvalue[i]=float(data[i])*stockquantity[i]
            totalvalue[i]=stockvalue[i]+cash[i]
        else:
            stockquantity[i]=(1-float(commissionrate))*cash[i-1]/float(data[i])
            cash[i]=0
            stockvalue[i]=float(data[i])*stockquantity[i]
            totalvalue[i]=stockvalue[i]+cash[i]
        i+=1
    return (totalvalue,cash,stockquantity,stockvalue)
    

In [ ]:
#This fuction will return the change (ratio) of the value in a list.
#For example, ratio([2,2,3,4]) equals [1,1,1.5,2]
def ratio(data):
    changeofratio=[]
    for item in data:
        changeofratio.append(float(item)/float(data[0]))
    return changeofratio

In [ ]:
#This is function is to genrate and compare the trends of two sets of data.
def compare(a,b):
    import matplotlib.pyplot as pyplot
    pyplot.plot(a,'g-',b,'r-')
    pyplot.xlabel('Day #')
    pyplot.ylabel('Ratio')

In [ ]:
#This function is to generate the trend of any array.
def trend(a):
    import matplotlib.pyplot as pyplot
    pyplot.plot(a)